In [1]:
from platform import python_version
import sys
print(sys.executable)
print(python_version())

/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10
3.10.10


In [2]:
import pandas as pd
import numpy as np
import jqfactor_analyzer as ja
import numba
import yfinance as yf
import backtrader as bk
import akshare as ak
import os
import scipy
import statsmodels.api as sm
import random
import math

In [3]:
os.environ.get('http_proxy')
os.environ.get('https_proxy')

'http://127.0.0.1:7890'

In [4]:
#pip freeze > requirements.txt

In [5]:
#data
futures_hq_subscribe_exchange_symbol_df = ak.futures_hq_subscribe_exchange_symbol()
print(futures_hq_subscribe_exchange_symbol_df)


      symbol  code
0   NYBOT-棉花    CT
1    LME镍3个月   NID
2    LME铅3个月   PBD
3    LME锡3个月   SND
4    LME锌3个月   ZSD
5    LME铝3个月   AHD
6    LME铜3个月   CAD
7    CBOT-黄豆     S
8    CBOT-小麦     W
9    CBOT-玉米     C
10  CBOT-黄豆油    BO
11  CBOT-黄豆粉    SM
12      日本橡胶   TRB
13    COMEX铜    HG
14  NYMEX天然气    NG
15   NYMEX原油    CL
16   COMEX白银    SI
17   COMEX黄金    GC
18   CME-瘦肉猪   LHC
19     布伦特原油   OIL
20       伦敦金   XAU
21       伦敦银   XAG
22      伦敦铂金   XPT
23      伦敦钯金   XPD
24       马棕油  FCPO
25     欧洲碳排放   EUA


In [6]:
'''
布伦特原油   OIL         SC
NYMEX天然气    NG       PG
伦敦金   XAU            AU
伦敦银   XAG            AG
LME镍3个月   NID
2    LME铅3个月   PBD
3    LME锡3个月   SND
4    LME锌3个月   ZSD
5    LME铝3个月   AHD
6    LME铜3个月   CAD   BC
'''

'\n布伦特原油   OIL         SC\nNYMEX天然气    NG       PG\n伦敦金   XAU            AU\n伦敦银   XAG            AG\nLME镍3个月   NID\n2    LME铅3个月   PBD\n3    LME锡3个月   SND\n4    LME锌3个月   ZSD\n5    LME铝3个月   AHD\n6    LME铜3个月   CAD   BC\n'

In [7]:
s1 = ['OIL','NG','XAU','XAG','CAD','PBD','NID','CAD',]
s2 = ['SC0','PG0','AU0','AG0','CU0','PB0','NI0','BC0',]

start_date = '2021-01-01'
end_date = '2023-08-10'

#@numba.jit(nopython=True)
def future_arb():
    for i in range(len(s1)):
        # if s1[i] not in ['OIL','CAD']:
        #     continue
        #corr ayasis
        futures_foreign_hist_df = ak.futures_foreign_hist(symbol=s1[i])
        futures_foreign_hist_df = futures_foreign_hist_df[(futures_foreign_hist_df.date >= start_date) & (futures_foreign_hist_df.date <= '2023-08-10')]
        date_range = pd.date_range(start=futures_foreign_hist_df["date"].min(), end=futures_foreign_hist_df["date"].max(), freq="D")  # freq="D"表示按天，可以按分钟，月，季度，年等
        futures_foreign_hist_df['date'] = pd.to_datetime(futures_foreign_hist_df['date'])
        futures_foreign_hist_df = futures_foreign_hist_df.set_index("date")
        futures_foreign_hist_df = futures_foreign_hist_df.resample("D").ffill()
        #print(futures_foreign_hist_df['close'])
        #print(futures_foreign_hist_df)
    
        #
        futures_zh_daily_sina_df = ak.futures_zh_daily_sina(symbol=s2[i])
        futures_zh_daily_sina_df = futures_zh_daily_sina_df[(futures_zh_daily_sina_df.date>=start_date) & (futures_zh_daily_sina_df.date <= '2023-08-10')]
        date_range = pd.date_range(start=futures_zh_daily_sina_df["date"].min(), end=futures_zh_daily_sina_df["date"].max(), freq="D")  # freq="D"表示按天，可以按分钟，月，季度，年等
        futures_zh_daily_sina_df['date'] = pd.to_datetime(futures_zh_daily_sina_df['date'])
        futures_zh_daily_sina_df = futures_zh_daily_sina_df.set_index("date")
        futures_zh_daily_sina_df = futures_zh_daily_sina_df.resample("D").ffill()
        
        #print(futures_zh_daily_sina_df)
        # print(s1[i])
        # print(len(futures_foreign_hist_df))
        # print(len(futures_zh_daily_sina_df))
        #print(np.array(futures_foreign_hist_df['close']))
        ## 计算Pearson相关系数

        # 进行ADF检验
        result = sm.tsa.coint(futures_zh_daily_sina_df['close'], futures_foreign_hist_df['close'])
        cointegration_statistic = result[0]
        p_value = result[1]
        
        print("cointegration_statistic:", cointegration_statistic)
        print("p-value:", p_value)

        #shift 1
        df_for = [futures_foreign_hist_df['close'][0] for i in range(1)] + futures_foreign_hist_df['close'][:-1].tolist()
        
        # pearson_corr, _ = scipy.stats.pearsonr(futures_zh_daily_sina_df['close'][10:], df_for[10:])
        # print("shifted %s-%s Pearson Correlation:" %(s1[i],s2[i]), pearson_corr)

        pearson_corr, _ = scipy.stats.pearsonr(futures_zh_daily_sina_df['close'], futures_foreign_hist_df['close'])
        print("%s-%s Pearson Correlation:" %(s1[i],s2[i]), pearson_corr,'\n')
future_arb()

cointegration_statistic: -1.03867942708734
p-value: 0.8952623048915642
OIL-SC0 Pearson Correlation: 0.9363947375489301 

cointegration_statistic: -3.231293144509903
p-value: 0.06484859510111797
NG-PG0 Pearson Correlation: 0.707845598860589 

cointegration_statistic: -0.4436139162388049
p-value: 0.9681120881734662
XAU-AU0 Pearson Correlation: 0.7280450192555985 

cointegration_statistic: 0.36969849478767564
p-value: 0.9916130880340316
XAG-AG0 Pearson Correlation: 0.857660187077641 

cointegration_statistic: -2.717362546790909
p-value: 0.19339629842201905
CAD-CU0 Pearson Correlation: 0.8998826197080161 

cointegration_statistic: -4.3800678885815225
p-value: 0.001914770484701523
PBD-PB0 Pearson Correlation: 0.5294509593092992 

cointegration_statistic: -2.5903643453104745
p-value: 0.2407318071019764
NID-NI0 Pearson Correlation: 0.7060881880308818 

cointegration_statistic: -2.8796864798399935
p-value: 0.14164155607175677
CAD-BC0 Pearson Correlation: 0.9057872567204117 



In [8]:
#ak.futures_display_main_sina()[symbol=='au0']

In [ ]:
import math
s1 = ['SC0','PG0','AU0','AG0','CU0','PB0','NI0','BC0','A0','B0','M0','JM0','J0','C0','CS0','MA0','PP0','CU0','RU0','Y0','P0',"AU0", "AG0", "CU0", "AL0", "ZN0", "PB0", "NI0", "SN0", "RB0", "HC0",
    "BU0", "RU0", "FU0", "SP0", "WR0", "SS0", "SA0", "UR0", "SR0", "CF0",
    "TA0", "MA0", "FG0", "RS0", "OI0",'AU0','AG0'
    "JR0", "SF0", "SM0", "AP0", "CJ0", "CY0", "RR0", "BB0", "FB0",
    "EB0", "V0", "L0", "PP0", "CS0", "JD0", "BU0", "HC0"]
#s2 = ['SC0','PG0','AU0','AG0','CU0','PB0','NI0','BC0','A0','B0','M0','JM0','J0','C0','CS0','MA0','PP0','CU0','RU0','Y0','P0']

#内盘
start_date = '2021-01-01'
end_date = '2023-08-10'

#@numba.jit(nopython=True)
def future_AA_arb():
    for i in range(1000000):
        futures_foreign_hist_df = []
        futures_zh_daily_sina_df = []
        # bc-cu,jm-j,rb-hc,cf-cy(x),wr-rb
        if i % 100 == 1:
            print(i)
        s11 = random.choice(s1)
        if s11 != 'C0':
            continue
        try:
            futures_foreign_hist_df = ak.futures_zh_daily_sina(symbol=s11)
        except:
            continue
        futures_foreign_hist_df = futures_foreign_hist_df[(futures_foreign_hist_df.date >= start_date) & (futures_foreign_hist_df.date <= '2023-08-10')]
        date_range = pd.date_range(start=futures_foreign_hist_df["date"].min(), end=futures_foreign_hist_df["date"].max(), freq="D")  # freq="D"表示按天，可以按分钟，月，季度，年等
        futures_foreign_hist_df['date'] = pd.to_datetime(futures_foreign_hist_df['date'])
        futures_foreign_hist_df = futures_foreign_hist_df.set_index("date")
        futures_foreign_hist_df = futures_foreign_hist_df.resample("D").ffill()
        #print(futures_foreign_hist_df['close'])
        #print(futures_foreign_hist_df)
    
        #
        s22 = random.choice(s1)
        if s22 == s11:
            continue
        try:
            futures_zh_daily_sina_df = ak.futures_zh_daily_sina(symbol=s22)
        except:
            continue
        futures_zh_daily_sina_df = futures_zh_daily_sina_df[(futures_zh_daily_sina_df.date>=start_date) & (futures_zh_daily_sina_df.date <= '2023-08-10')]
        date_range = pd.date_range(start=futures_zh_daily_sina_df["date"].min(), end=futures_zh_daily_sina_df["date"].max(), freq="D")  # freq="D"表示按天，可以按分钟，月，季度，年等
        futures_zh_daily_sina_df['date'] = pd.to_datetime(futures_zh_daily_sina_df['date'])
        futures_zh_daily_sina_df = futures_zh_daily_sina_df.set_index("date")
        futures_zh_daily_sina_df = futures_zh_daily_sina_df.resample("D").ffill()
        
        #print(futures_zh_daily_sina_df)
        # print(s1[i])
        # print(len(futures_foreign_hist_df))
        # print(len(futures_zh_daily_sina_df))
        #print(np.array(futures_foreign_hist_df['close']))
        ## 计算Pearson相关系数

        #shift 1
        df_for = [futures_foreign_hist_df['close'][0] for i in range(1)] + futures_foreign_hist_df['close'][:-1].tolist()
        
        # pearson_corr, _ = scipy.stats.pearsonr(futures_zh_daily_sina_df['close'][10:], df_for[10:])
        # print("shifted %s-%s Pearson Correlation:" %(s1[i],s2[i]), pearson_corr)
        try:
            pearson_corr, _ = scipy.stats.pearsonr(futures_zh_daily_sina_df['close'], futures_foreign_hist_df['close'])
        except:
            continue
        if abs(pearson_corr) > 0.5:
            print("%s-%s Pearson Correlation:" %(s11,s22), pearson_corr)
                 # 进行ADF检验
            try:
                # result = sm.tsa.coint(futures_zh_daily_sina_df['close'], futures_foreign_hist_df['close'])
                # cointegration_statistic = result[0]
                # p_value = result[1]
                
                # print("cointegration_statistic:", cointegration_statistic)
                # print("p-value:", p_value,'\n')
                print("sgima",np.std(futures_foreign_hist_df['close']-futures_zh_daily_sina_df['close']))
            except:
                continue
future_AA_arb()

1
101
201
C0-M0 Pearson Correlation: 0.5927683688100996
sgima 270.76684169180925
301
401
C0-CS0 Pearson Correlation: 0.5338828270006445
sgima 134.6743996230983
501
C0-NI0 Pearson Correlation: 0.6362825401086697
sgima 31432.59614814465
C0-ZN0 Pearson Correlation: 0.5119175694551216
sgima 1877.088256930307
601
701
801
901
1001
1101
1201
1301
1401
1501
C0-CS0 Pearson Correlation: 0.5338828270006445
sgima 134.6743996230983
1601
1701
1801
1901
C0-ZN0 Pearson Correlation: 0.5119175694551216
sgima 1877.088256930307
C0-CS0 Pearson Correlation: 0.5338828270006445
sgima 134.6743996230983
2001
2101
2201
2301
2401
2501
2601
2701
2801
C0-CS0 Pearson Correlation: 0.5338828270006445
sgima 134.6743996230983
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
C0-NI0 Pearson Correlation: 0.6362825401086697
sgima 31432.59614814465
3901
4001
4101
4201
4301
4401
C0-NI0 Pearson Correlation: 0.6362825401086697
sgima 31432.59614814465
4501
4601
C0-M0 Pearson Correlation: 0.5927683688100996
sgima 270.76684169180